In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv
/kaggle/input/word2vec/pytorch/default/1/word2vec (1).model
/kaggle/input/preprocessed-sentiment-data/preprocessed_sentiment_data.csv


# Data

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
df=pd.read_csv("/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")

In [4]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [6]:
df['review'][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [7]:
df['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [8]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [10]:
df.duplicated().sum()

418

In [11]:
df.drop_duplicates(inplace=True)

In [12]:
df.duplicated().sum()

0

# Text Preprocessing

In [3]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [15]:
def preprocess_text(text):
    text = text.lower()  # Lowercasing
    text = re.sub(r'https?://\S+|www\.\S+', '', text)  # Remove URLs
    text = re.sub(re.compile('<.*?>'), '', text)  # tags
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters
    tokens = word_tokenize(text)  # Tokenization
    tokens = [word for word in tokens if word not in stopwords.words('english')]  # Stopword removal
    return " ".join(tokens)

df['cleaned_text'] = df['review'].apply(preprocess_text)


In [16]:
df

,review,sentiment,cleaned_text
0,One of the other reviewers has mentioned that ...,positive,one reviewers mentioned watching oz episode yo...
1,A wonderful little production. <br /><br />The...,positive,wonderful little production filming technique ...
2,I thought this was a wonderful way to spend ti...,positive,thought wonderful way spend time hot summer we...
3,Basically there's a family where a little boy ...,negative,basically theres family little boy jake thinks...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,petter matteis love time money visually stunni...
...,...,...,...
49995,I thought this movie did a down right good job...,positive,thought movie right good job wasnt creative or...
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative,bad plot bad dialogue bad acting idiotic direc...
49997,I am a Catholic taught in parochial elementary...,negative,catholic taught parochial elementary schools n...
49998,I'm going to have to disagree with the previou...,negative,im going disagree previous comment side maltin...


In [17]:
import spacy

nlp = spacy.load("en_core_web_sm")

def lemmass(text):
    doc = nlp(text)
    lemmas = [token.lemma_ for token in doc]
    return ' '.join(lemmas)

df['cleaned_text'] = df['cleaned_text'].apply(lemmass)

In [18]:
df=df[['cleaned_text','sentiment']]
df

,cleaned_text,sentiment
0,one reviewer mention watch oz episode you ll h...,positive
1,wonderful little production filming technique ...,positive
2,think wonderful way spend time hot summer week...,positive
3,basically there s family little boy jake think...,negative
4,petter matteis love time money visually stunni...,positive
...,...,...
49995,think movie right good job be not creative ori...,positive
49996,bad plot bad dialogue bad act idiotic direct a...,negative
49997,catholic teach parochial elementary school nun...,negative
49998,I m go disagree previous comment side maltin o...,negative


# Save & Load Preprocessed Data

In [ ]:
df.to_csv("preprocessed_sentiment_data.csv", index=False)

In [4]:
df = pd.read_csv("/kaggle/input/preprocessed-sentiment-data/preprocessed_sentiment_data.csv")

# Train and Test

In [5]:
X = df.iloc[:,0:1]
y = df['sentiment']

print(X)
print(y)

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y = encoder.fit_transform(y)
print(y)

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

                                            cleaned_text
0      one reviewer mention watch oz episode you ll h...
1      wonderful little production filming technique ...
2      think wonderful way spend time hot summer week...
3      basically there s family little boy jake think...
4      petter matteis love time money visually stunni...
...                                                  ...
49577  think movie right good job be not creative ori...
49578  bad plot bad dialogue bad act idiotic direct a...
49579  catholic teach parochial elementary school nun...
49580  I m go disagree previous comment side maltin o...
49581  one expect star trek movie high art fan expect...

[49582 rows x 1 columns]
0        positive
1        positive
2        positive
3        negative
4        positive
           ...   
49577    positive
49578    negative
49579    negative
49580    negative
49581    negative
Name: sentiment, Length: 49582, dtype: object
[1 1 1 ... 0 0 0]
(39665, 1) (9917, 1) (39665,

# Convert Text to Vectors

### 1. Bag of Words (BOW)

In [40]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=10000)
X_train_bow1 = cv.fit_transform(X_train['cleaned_text']).toarray()
X_test_bow1 = cv.transform(X_test['cleaned_text']).toarray()

In [41]:
X_train_bow1.shape,X_test_bow1.shape

((39665, 10000), (9917, 10000))

### 2. TfIdf

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=10000)
X_train_tfidf1 = tfidf.fit_transform(X_train['cleaned_text'])
X_test_tfidf1 = tfidf.transform(X_test['cleaned_text'])

In [7]:
X_train_tfidf1.shape,X_test_tfidf1.shape

((39665, 10000), (9917, 10000))

### 3. Word2vec

In [5]:
import gensim
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

In [6]:
tokenized_corpus = [word_tokenize(sentence.lower()) for sentence in X_train['cleaned_text']]

word2vec_model = Word2Vec(sentences=tokenized_corpus, vector_size=300, window=5, min_count=1, workers=4)

In [7]:
print(word2vec_model.wv['watch'])  # Example word embeddin


[ 0.02491953 -1.0913501   0.14735903 -0.6497404   1.3276836   0.5583584
  0.9968599   0.70203054  0.52654666 -0.22924031  0.00974168  0.34475282
 -1.7714472  -0.41322938 -1.0833504  -0.1131838   1.3735849   0.398398
  1.6907634  -0.9985803   0.2885862   0.06600291  0.49371925  0.01361038
 -0.26424694  0.72129405 -0.74179727 -0.35931048  0.7354718   0.4325307
  0.7961574  -0.22549355  0.96308523  0.15833628 -0.60517365 -0.6502583
 -2.5667667  -1.4583397   0.5311674   0.10063814  0.0450785  -0.2851342
  1.1798848  -1.179137   -1.0731796  -0.19313093 -0.08611448  0.70088345
 -0.5242068   0.87944776 -0.03869414  0.8254978   0.57637036  0.36325517
  0.8194674   0.79182553  0.20090923 -0.09786388  0.78275347 -0.00733812
 -0.37132046 -0.80092144  0.11803232  1.6788431  -0.81999207  0.2745788
  0.79905534 -0.13191465 -1.7405558  -0.13199843 -0.39005795  0.63273585
  0.39711633 -0.66656566  0.70931846  0.8683811   0.49707744  0.15708178
 -0.8089754  -0.3290382  -1.0001711   1.2454933  -1.203178

In [8]:
word2vec_model.save("word2vec.model")

In [ ]:
word2vec_model = Word2Vec.load("/kaggle/input/word2vec/pytorch/default/1/word2vec(1).model")

In [14]:
import numpy as np

def sentence_to_vector(sentence, model, vector_size=300):
    words = word_tokenize(sentence.lower())  
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    
    if len(word_vectors) == 0:
        return np.zeros(vector_size) 


    return np.mean(word_vectors, axis=0) 

In [15]:
import numpy as np
from tqdm import tqdm  

X_train_word2vec = np.array([sentence_to_vector(text, word2vec_model) for text in tqdm(X_train["cleaned_text"])])
X_test_word2vec = np.array([sentence_to_vector(text, word2vec_model) for text in tqdm(X_test["cleaned_text"])])


100%|██████████| 9917/9917 [00:07<00:00, 1242.75it/s]


In [16]:
X_train_word2vec.shape,X_test_word2vec.shape

((39665, 300), (9917, 300))

# Models

### 1. Naive bayes

In [77]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report

In [16]:
nb_model = MultinomialNB()
nb_model.fit(X_train_bow1, y_train)

y_pred = nb_model.predict(X_test_bow1)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8437027326812544
Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.85      0.85      5033
           1       0.85      0.84      0.84      4884

    accuracy                           0.84      9917
   macro avg       0.84      0.84      0.84      9917
weighted avg       0.84      0.84      0.84      9917



In [8]:
nb_model = MultinomialNB()
nb_model.fit(X_train_tfidf1, y_train)

y_pred = nb_model.predict(X_test_tfidf1)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8531814056670364
Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.85      0.85      5033
           1       0.85      0.86      0.85      4884

    accuracy                           0.85      9917
   macro avg       0.85      0.85      0.85      9917
weighted avg       0.85      0.85      0.85      9917



In [15]:
gnb_model = GaussianNB()
gnb_model.fit(X_train_bow1, y_train)

y_pred = gnb_model.predict(X_test_bow1)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.7137239084400524
Classification Report:
               precision    recall  f1-score   support

           0       0.67      0.87      0.75      5033
           1       0.80      0.56      0.66      4884

    accuracy                           0.71      9917
   macro avg       0.73      0.71      0.71      9917
weighted avg       0.73      0.71      0.71      9917



In [14]:
X_train_dense = X_train_tfidf1
X_test_dense = X_test_tfidf1.toarray()
gnb_model = GaussianNB()
gnb_model.fit(X_train_dense, y_train)

y_pred = gnb_model.predict(X_test_dense)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.7770495109408088
Classification Report:
               precision    recall  f1-score   support

           0       0.76      0.81      0.79      5033
           1       0.79      0.74      0.77      4884

    accuracy                           0.78      9917
   macro avg       0.78      0.78      0.78      9917
weighted avg       0.78      0.78      0.78      9917



In [81]:
gnb_model = GaussianNB()
gnb_model.fit(X_train_word2vec, y_train)

y_pred = gnb_model.predict(X_test_word2vec)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.7544620348895835
Classification Report:
               precision    recall  f1-score   support

           0       0.76      0.76      0.76      5033
           1       0.75      0.75      0.75      4884

    accuracy                           0.75      9917
   macro avg       0.75      0.75      0.75      9917
weighted avg       0.75      0.75      0.75      9917



### 2. Random Forest

In [82]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [18]:
rf_model = RandomForestClassifier()
rf_model.fit(X_train_bow1, y_train)

y_pred = rf_model.predict(X_test_bow1)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8432993848946254
Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.84      0.85      5033
           1       0.84      0.84      0.84      4884

    accuracy                           0.84      9917
   macro avg       0.84      0.84      0.84      9917
weighted avg       0.84      0.84      0.84      9917



In [12]:
rf_model = RandomForestClassifier()
rf_model.fit(X_train_tfidf1, y_train)

y_pred = rf_model.predict(X_test_tfidf1)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8428960371079963
Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.84      0.84      5033
           1       0.84      0.84      0.84      4884

    accuracy                           0.84      9917
   macro avg       0.84      0.84      0.84      9917
weighted avg       0.84      0.84      0.84      9917



In [83]:
rf_model = RandomForestClassifier()
rf_model.fit(X_train_word2vec, y_train)

y_pred = rf_model.predict(X_test_word2vec)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8255520822829485
Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.80      0.82      5033
           1       0.81      0.85      0.83      4884

    accuracy                           0.83      9917
   macro avg       0.83      0.83      0.83      9917
weighted avg       0.83      0.83      0.83      9917



### 3. XGBoost

In [9]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report

In [13]:
xgb_model = xgb.XGBClassifier()
xgb_model.fit(X_train_bow1, y_train)

# Predictions
y_pred = xgb_model.predict(X_test_bow1)

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8511646667338914
Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.82      0.85      5033
           1       0.83      0.88      0.85      4884

    accuracy                           0.85      9917
   macro avg       0.85      0.85      0.85      9917
weighted avg       0.85      0.85      0.85      9917



In [10]:
xgb_model = xgb.XGBClassifier()
xgb_model.fit(X_train_tfidf1, y_train)

y_pred = xgb_model.predict(X_test_tfidf1)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8541897751336089
Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.83      0.85      5033
           1       0.83      0.88      0.86      4884

    accuracy                           0.85      9917
   macro avg       0.85      0.85      0.85      9917
weighted avg       0.86      0.85      0.85      9917



In [85]:
xgb_model = xgb.XGBClassifier()
xgb_model.fit(X_train_word2vec, y_train)

# Predictions
y_pred = xgb_model.predict(X_test_word2vec)

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8436018957345972
Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.83      0.84      5033
           1       0.83      0.86      0.84      4884

    accuracy                           0.84      9917
   macro avg       0.84      0.84      0.84      9917
weighted avg       0.84      0.84      0.84      9917



### 4. CatBoost

In [86]:
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, classification_report

In [23]:
cat_model = CatBoostClassifier(verbose=False)
cat_model.fit(X_train_bow1, y_train)

y_pred = cat_model.predict(X_test_bow1)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8710295452253706
Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.85      0.87      5033
           1       0.85      0.90      0.87      4884

    accuracy                           0.87      9917
   macro avg       0.87      0.87      0.87      9917
weighted avg       0.87      0.87      0.87      9917



In [21]:
cat_model = CatBoostClassifier(verbose=False)
cat_model.fit(X_train_tfidf1, y_train)

y_pred = cat_model.predict(X_test_tfidf1)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8662902087324795
Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.84      0.86      5033
           1       0.85      0.89      0.87      4884

    accuracy                           0.87      9917
   macro avg       0.87      0.87      0.87      9917
weighted avg       0.87      0.87      0.87      9917



In [87]:
cat_model = CatBoostClassifier(verbose=False)
cat_model.fit(X_train_word2vec, y_train)

y_pred = cat_model.predict(X_test_word2vec)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8574165574266411
Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.84      0.86      5033
           1       0.84      0.87      0.86      4884

    accuracy                           0.86      9917
   macro avg       0.86      0.86      0.86      9917
weighted avg       0.86      0.86      0.86      9917



### 5. Lightgbm

In [89]:
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, classification_report

In [27]:
lgb_model = LGBMClassifier(n_jobs=-1,verbose=0)
lgb_model.fit(X_train_bow1, y_train)

y_pred = lgb_model.predict(X_test_bow1)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.8561056771200968
Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.83      0.85      5033
           1       0.84      0.88      0.86      4884

    accuracy                           0.86      9917
   macro avg       0.86      0.86      0.86      9917
weighted avg       0.86      0.86      0.86      9917



In [27]:
lgb_model = LGBMClassifier(n_jobs=-1,verbose=0)
lgb_model.fit(X_train_tfidf1, y_train)

y_pred = lgb_model.predict(X_test_tfidf1)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8566098618533831
Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.84      0.86      5033
           1       0.84      0.88      0.86      4884

    accuracy                           0.86      9917
   macro avg       0.86      0.86      0.86      9917
weighted avg       0.86      0.86      0.86      9917



In [90]:
lgb_model = LGBMClassifier(n_jobs=-1,verbose=0)
lgb_model.fit(X_train_word2vec, y_train)

y_pred = lgb_model.predict(X_test_word2vec)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.845517797721085
Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.83      0.84      5033
           1       0.83      0.86      0.85      4884

    accuracy                           0.85      9917
   macro avg       0.85      0.85      0.85      9917
weighted avg       0.85      0.85      0.85      9917



### 6. ANN

In [8]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset, DataLoader
from torchvision import datasets
from torchvision import transforms
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import random_split
import torchinfo

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [51]:
X_train_tensor_bow = torch.tensor(X_train_bow1, dtype=torch.float32).to(device)
Y_train_tensor_bow = torch.tensor(y_train, dtype=torch.float32).to(device) 

train_dataset = TensorDataset(X_train_tensor_bow, Y_train_tensor_bow)

train_dataloader_bow = DataLoader(train_dataset, batch_size=32, shuffle=True)

for batch in train_dataloader_bow:
    X_batch, Y_batch = batch
    print("X_batch shape:", X_batch.shape)
    print("Y_batch shape:", Y_batch.shape)
    break  # Only print first batch
print(len(train_dataloader))

X_test_tensor_bow = torch.tensor(X_test_bow1, dtype=torch.float32).to(device) 
Y_test_tensor_bow = torch.tensor(y_test, dtype=torch.float32).to(device)  

test_dataset = TensorDataset(X_test_tensor_bow, Y_test_tensor_bow)

test_dataloader_bow = DataLoader(test_dataset, batch_size=32, shuffle=True)

for batch in test_dataloader_bow:
    X_batch, Y_batch = batch
    print("X_batch shape:", X_batch.shape)
    print("Y_batch shape:", Y_batch.shape)
    break  # Only print first batch
print(len(test_dataloader))

X_batch shape: torch.Size([32, 10000])
Y_batch shape: torch.Size([32])
1240
X_batch shape: torch.Size([32, 10000])
Y_batch shape: torch.Size([32])
310


In [11]:
X_train_dense = X_train_tfidf1.toarray()  
X_test_dense = X_test_tfidf1.toarray()

In [65]:
X_train_tensor_tfidf = torch.tensor(X_train_dense, dtype=torch.float32).to(device) 
Y_train_tensor_tfidf = torch.tensor(y_train, dtype=torch.float32).to(device)

train_dataset = TensorDataset(X_train_tensor_tfidf, Y_train_tensor_tfidf)

train_dataloader_tfidf = DataLoader(train_dataset, batch_size=32, shuffle=True)

for batch in train_dataloader_tfidf:
    X_batch, Y_batch = batch
    print("X_batch shape:", X_batch.shape)
    print("Y_batch shape:", Y_batch.shape)
    break  # Only print first batch
print(len(train_dataloader_tfidf))

X_test_tensor_tfidf = torch.tensor(X_test_dense, dtype=torch.float32).to(device)
Y_test_tensor_tfidf = torch.tensor(y_test, dtype=torch.float32).to(device) 

test_dataset = TensorDataset(X_test_tensor_tfidf, Y_test_tensor_tfidf)

test_dataloader_tfidf = DataLoader(test_dataset, batch_size=32, shuffle=False)

for batch in test_dataloader_tfidf:
    X_batch, Y_batch = batch
    print("X_batch shape:", X_batch.shape)
    print("Y_batch shape:", Y_batch.shape)
    break  # Only print first batch
print(len(test_dataloader_tfidf))

X_batch shape: torch.Size([32, 10000])
Y_batch shape: torch.Size([32])
1240
X_batch shape: torch.Size([32, 10000])
Y_batch shape: torch.Size([32])
310


In [93]:
X_train_tensor_word2vec = torch.tensor(X_train_word2vec, dtype=torch.float32).to(device) 
Y_train_tensor_word2vec = torch.tensor(y_train, dtype=torch.float32).to(device)

train_dataset = TensorDataset(X_train_tensor_word2vec, Y_train_tensor_tfidf)

train_dataloader_word2vec = DataLoader(train_dataset, batch_size=32, shuffle=True)

for batch in train_dataloader_word2vec:
    X_batch, Y_batch = batch
    print("X_batch shape:", X_batch.shape)
    print("Y_batch shape:", Y_batch.shape)
    break  # Only print first batch
print(len(train_dataloader_word2vec))

X_test_tensor_word2vec = torch.tensor(X_test_word2vec, dtype=torch.float32).to(device)
Y_test_tensor_word2vec = torch.tensor(y_test, dtype=torch.float32).to(device) 

test_dataset = TensorDataset(X_test_tensor_word2vec, Y_test_tensor_tfidf)

test_dataloader_word2vec = DataLoader(test_dataset, batch_size=32, shuffle=False)

for batch in test_dataloader_word2vec:
    X_batch, Y_batch = batch
    print("X_batch shape:", X_batch.shape)
    print("Y_batch shape:", Y_batch.shape)
    break  # Only print first batch
print(len(test_dataloader_word2vec))

X_batch shape: torch.Size([32, 300])
Y_batch shape: torch.Size([32])
1240
X_batch shape: torch.Size([32, 300])
Y_batch shape: torch.Size([32])
310


In [66]:
class ANN(nn.Module):
    def __init__(self):
        super(ANN, self).__init__()
        
        self.seq=nn.Sequential(
            nn.Linear(in_features=10000,out_features=2000),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(in_features=2000,out_features=1),
        )

    def forward(self,x):
        return self.seq(x)
  
mymodel=ANN().to(device)

print("Summary of the architecture (batch_size=32):")
torchinfo.summary(mymodel, input_size=(32,10000))

Summary of the architecture (batch_size=32):


Layer (type:depth-idx)                   Output Shape              Param #
ANN                                      [32, 1]                   --
├─Sequential: 1-1                        [32, 1]                   --
│    └─Linear: 2-1                       [32, 2000]                20,002,000
│    └─ReLU: 2-2                         [32, 2000]                --
│    └─Dropout: 2-3                      [32, 2000]                --
│    └─Linear: 2-4                       [32, 1]                   2,001
Total params: 20,004,001
Trainable params: 20,004,001
Non-trainable params: 0
Total mult-adds (M): 640.13
Input size (MB): 1.28
Forward/backward pass size (MB): 0.51
Params size (MB): 80.02
Estimated Total Size (MB): 81.81

In [67]:
loss_fn = nn.BCEWithLogitsLoss()  # Binary Cross-Entropy Loss
optimizer=torch.optim.Adam(params=mymodel.parameters(),lr=0.01, weight_decay=1e-5)

In [100]:
class ANN(nn.Module):
    def __init__(self):
        super(ANN, self).__init__()
        
        self.seq=nn.Sequential(
            nn.Linear(in_features=300,out_features=200),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(in_features=200,out_features=100),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(in_features=100,out_features=10),
            nn.ReLU(),
            nn.Linear(in_features=10,out_features=1)
        )

    def forward(self,x):
        return self.seq(x)
  
mymodel=ANN().to(device)

print("Summary of the architecture (batch_size=32):")
torchinfo.summary(mymodel, input_size=(32,300))

Summary of the architecture (batch_size=32):


Layer (type:depth-idx)                   Output Shape              Param #
ANN                                      [32, 1]                   --
├─Sequential: 1-1                        [32, 1]                   --
│    └─Linear: 2-1                       [32, 200]                 60,200
│    └─ReLU: 2-2                         [32, 200]                 --
│    └─Dropout: 2-3                      [32, 200]                 --
│    └─Linear: 2-4                       [32, 100]                 20,100
│    └─ReLU: 2-5                         [32, 100]                 --
│    └─Dropout: 2-6                      [32, 100]                 --
│    └─Linear: 2-7                       [32, 10]                  1,010
│    └─ReLU: 2-8                         [32, 10]                  --
│    └─Linear: 2-9                       [32, 1]                   11
Total params: 81,321
Trainable params: 81,321
Non-trainable params: 0
Total mult-adds (M): 2.60
Input size (MB): 0.04
Forward/backward pass size

In [101]:
loss_fn = nn.BCEWithLogitsLoss()  # Binary Cross-Entropy Loss
optimizer=torch.optim.Adam(params=mymodel.parameters(),lr=0.01, weight_decay=1e-5)

In [102]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.01):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = float('inf')
        self.counter = 0

    def __call__(self, val_loss):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1

        return self.counter >= self.patience

In [29]:
def accuracy_fn(y_true, y_pred):
    y_true = y_true.clone().cpu()
    y_pred = y_pred.clone().cpu()

    correct = torch.eq(y_true, y_pred).sum().item() 
    acc = (correct / len(y_pred)) * 100 
    return acc

In [59]:
def train_step(model,data_loader,loss_fn,optimizer:torch,accuracy_fn):
  train_loss,train_acc=0,0
  
  for X,y in data_loader:
      X, y = X.to(device), y.to(device)
      
      y_logits=model(X).squeeze()
      y_pred_probs = torch.sigmoid(y_logits)
      y_preds = torch.round(y_pred_probs)

      loss=loss_fn(y_logits,y)
      train_loss=train_loss+loss.item()
      train_acc+=accuracy_fn(y_true=y.detach(),y_pred=y_preds.detach())
      
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
  
  train_loss/=len(data_loader)
  train_acc/=len(data_loader)
  print(f"Train loss: {train_loss:.5f} | Train accuracy: {train_acc:.2f}%")
  return train_loss

def test_step(model,data_loader,loss_fn,accuracy_fn):
  test_loss,test_acc=0,0

  model.eval()

  with torch.inference_mode():
    for X,y in data_loader:
        X, y = X.to(device), y.to(device) 
        
        y_logits=model(X).squeeze()
        y_pred_probs = torch.sigmoid(y_logits)
        y_preds = torch.round(y_pred_probs)
            
        test_loss += loss_fn(y_logits, y).item()
        test_acc += accuracy_fn(y_true=y.detach(),y_pred=y_preds.detach())
    
    test_loss /= len(data_loader)
    test_acc /= len(data_loader)
    print(f"Test loss: {test_loss:.5f} | Test accuracy: {test_acc:.2f}%\n")
    return test_loss

### BOW

In [64]:
torch.manual_seed(42)
epochs=30
early_stopping = EarlyStopping()
for epoch in range(epochs):
  print("epoch :",epoch+1)
  train=train_step(model=mymodel,
             data_loader=train_dataloader_bow,
             loss_fn=loss_fn,
             optimizer=optimizer,
             accuracy_fn=accuracy_fn)
  
  test=test_step(model=mymodel,
            data_loader=test_dataloader_bow,
            loss_fn=loss_fn,
            accuracy_fn=accuracy_fn)

  if early_stopping(test):
      print(f"Training halted at epoch {epoch + 1}")
      break

epoch : 1
Train loss: 0.19051 | Train accuracy: 93.46%
Test loss: 0.52701 | Test accuracy: 85.23%

epoch : 2
Train loss: 0.18629 | Train accuracy: 93.41%
Test loss: 0.40154 | Test accuracy: 86.31%

epoch : 3
Train loss: 0.12923 | Train accuracy: 95.60%
Test loss: 0.57214 | Test accuracy: 85.52%

epoch : 4
Train loss: 0.15076 | Train accuracy: 95.04%
Test loss: 0.55297 | Test accuracy: 84.94%

epoch : 5
Train loss: 0.12384 | Train accuracy: 95.99%
Test loss: 0.58548 | Test accuracy: 85.21%

epoch : 6
Train loss: 0.11620 | Train accuracy: 96.38%
Test loss: 0.65738 | Test accuracy: 85.14%

epoch : 7
Train loss: 0.10244 | Train accuracy: 96.68%
Test loss: 0.66888 | Test accuracy: 84.98%

Training halted at epoch 7


### TFIDF

In [69]:
torch.manual_seed(42)
epochs=30
early_stopping = EarlyStopping()
for epoch in range(epochs):
  print("epoch :",epoch+1)
  train=train_step(model=mymodel,
             data_loader=train_dataloader_tfidf,
             loss_fn=loss_fn,
             optimizer=optimizer,
             accuracy_fn=accuracy_fn)
  
  test=test_step(model=mymodel,
            data_loader=test_dataloader_tfidf,
            loss_fn=loss_fn,
            accuracy_fn=accuracy_fn)

  if early_stopping(test):
      print(f"Training halted at epoch {epoch + 1}")
      break

epoch : 1
Train loss: 0.32935 | Train accuracy: 85.80%
Test loss: 0.28462 | Test accuracy: 87.77%

epoch : 2
Train loss: 0.26338 | Train accuracy: 89.18%
Test loss: 0.27949 | Test accuracy: 88.20%

epoch : 3
Train loss: 0.22324 | Train accuracy: 90.96%
Test loss: 0.28201 | Test accuracy: 88.01%

epoch : 4
Train loss: 0.16191 | Train accuracy: 93.82%
Test loss: 0.31129 | Test accuracy: 87.57%

epoch : 5
Train loss: 0.10627 | Train accuracy: 96.06%
Test loss: 0.36732 | Test accuracy: 86.96%

epoch : 6
Train loss: 0.07748 | Train accuracy: 97.19%
Test loss: 0.40354 | Test accuracy: 86.48%

Training halted at epoch 6


### word2vec

In [103]:
torch.manual_seed(42)
epochs=30
early_stopping = EarlyStopping()
for epoch in range(epochs):
  print("epoch :",epoch+1)
  train=train_step(model=mymodel,
             data_loader=train_dataloader_word2vec,
             loss_fn=loss_fn,
             optimizer=optimizer,
             accuracy_fn=accuracy_fn)
  
  test=test_step(model=mymodel,
            data_loader=test_dataloader_word2vec,
            loss_fn=loss_fn,
            accuracy_fn=accuracy_fn)

  if early_stopping(test):
      print(f"Training halted at epoch {epoch + 1}")
      break

epoch : 1
Train loss: 0.44173 | Train accuracy: 80.37%
Test loss: 0.36077 | Test accuracy: 83.72%

epoch : 2
Train loss: 0.35838 | Train accuracy: 84.53%
Test loss: 0.35319 | Test accuracy: 85.04%

epoch : 3
Train loss: 0.35407 | Train accuracy: 84.86%
Test loss: 0.33549 | Test accuracy: 85.24%

epoch : 4
Train loss: 0.35059 | Train accuracy: 84.92%
Test loss: 0.34099 | Test accuracy: 85.49%

epoch : 5
Train loss: 0.34862 | Train accuracy: 84.94%
Test loss: 0.33116 | Test accuracy: 85.74%

epoch : 6
Train loss: 0.34211 | Train accuracy: 85.21%
Test loss: 0.34015 | Test accuracy: 85.76%

epoch : 7
Train loss: 0.34140 | Train accuracy: 85.14%
Test loss: 0.35405 | Test accuracy: 84.99%

epoch : 8
Train loss: 0.34178 | Train accuracy: 85.22%
Test loss: 0.33331 | Test accuracy: 85.09%

Training halted at epoch 8


### 7. LSTM

In [9]:
MAX_LEN = 100

def text_to_embedding(text, word2vec_model, max_len=MAX_LEN):
    words = text.split() 
    embeddings = []

    for word in words:
        if word in word2vec_model.wv:
            embeddings.append(word2vec_model.wv[word])
        else:
            embeddings.append(np.zeros(300))

    if len(embeddings) < max_len:
        embeddings += [np.zeros(300)] * (max_len - len(embeddings))
    else:
        embeddings = embeddings[:max_len]

    return np.array(embeddings)

X_train_embeddings = np.array([text_to_embedding(text, word2vec_model) for text in X_train['cleaned_text']])
X_test_embeddings = np.array([text_to_embedding(text, word2vec_model) for text in X_test['cleaned_text']])

X_train_tensor = torch.tensor(X_train_embeddings, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_embeddings, dtype=torch.float32)

In [12]:
import gc

del X_train_embeddings
del X_test_embeddings

gc.collect()

2799

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [17]:
X_train_tensor_word2vec = X_train_tensor.to(device) 
Y_train_tensor_word2vec = torch.tensor(y_train, dtype=torch.float32).to(device)

train_dataset = TensorDataset(X_train_tensor_word2vec, Y_train_tensor_word2vec)

train_dataloader_word2vec = DataLoader(train_dataset, batch_size=32, shuffle=True)

for batch in train_dataloader_word2vec:
    X_batch, Y_batch = batch
    print("X_batch shape:", X_batch.shape)
    print("Y_batch shape:", Y_batch.shape)
    break  # Only print first batch
print(len(train_dataloader_word2vec))

X_test_tensor_word2vec = X_test_tensor.to(device)
Y_test_tensor_word2vec = torch.tensor(y_test, dtype=torch.float32).to(device) 

test_dataset = TensorDataset(X_test_tensor_word2vec, Y_test_tensor_word2vec)

test_dataloader_word2vec = DataLoader(test_dataset, batch_size=32, shuffle=True)

for batch in test_dataloader_word2vec:
    X_batch, Y_batch = batch
    print("X_batch shape:", X_batch.shape)
    print("Y_batch shape:", Y_batch.shape)
    break  # Only print first batch
print(len(test_dataloader_word2vec))

X_batch shape: torch.Size([32, 100, 300])
Y_batch shape: torch.Size([32])
1240
X_batch shape: torch.Size([32, 100, 300])
Y_batch shape: torch.Size([32])
310


In [71]:
class LSTM(nn.Module):
    def __init__(self):
        super().__init__()

        self.lstm1=nn.LSTM(input_size=300,hidden_size=50,num_layers=3,batch_first=True,bidirectional=True)
        self.fc = nn.Linear(50*2 ,1)

    def forward(self,x):
        lstm_out,_ = self.lstm1(x)
        out = self.fc(lstm_out[:, -1, :])  # Last hidden state
        return out


mymodel = LSTM().to(device)
        

print("Summary of the architecture (batch_size=32):")
torchinfo.summary(mymodel, input_size=(32,100,300))

Summary of the architecture (batch_size=32):


Layer (type:depth-idx)                   Output Shape              Param #
LSTM                                     [32, 1]                   --
├─LSTM: 1-1                              [32, 100, 100]            262,400
├─Linear: 1-2                            [32, 1]                   101
Total params: 262,501
Trainable params: 262,501
Non-trainable params: 0
Total mult-adds (M): 839.68
Input size (MB): 3.84
Forward/backward pass size (MB): 2.56
Params size (MB): 1.05
Estimated Total Size (MB): 7.45

In [72]:
loss_fn = nn.BCEWithLogitsLoss()  # Binary Cross-Entropy Loss
optimizer=torch.optim.Adam(params=mymodel.parameters(),lr=0.01, weight_decay=1e-5)

In [55]:
def accuracy_fn(y_true, y_pred):
    y_true = y_true.clone().cpu()
    y_pred = y_pred.clone().cpu()

    correct = torch.eq(y_true, y_pred).sum().item() 
    acc = (correct / len(y_pred)) * 100 
    return acc

In [56]:
def train_step(model,data_loader,loss_fn,optimizer:torch,accuracy_fn):
  model.train()
  train_loss,train_acc=0,0
  
  for X,y in data_loader:
      X, y = X.to(device), y.to(device)
      
      y_logits=model(X).squeeze()
      y_pred_probs = torch.sigmoid(y_logits)
      y_preds = torch.round(y_pred_probs)

      loss=loss_fn(y_logits,y)
      train_loss=train_loss+loss.item()
      train_acc+=accuracy_fn(y_true=y.detach(),y_pred=y_preds.detach())
      
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
  
  train_loss/=len(data_loader)
  train_acc/=len(data_loader)
  print(f"Train loss: {train_loss:.5f} | Train accuracy: {train_acc:.2f}%")
  return train_loss

def test_step(model,data_loader,loss_fn,accuracy_fn):
  test_loss,test_acc=0,0

  model.eval()

  with torch.inference_mode():
    for X,y in data_loader:
        X, y = X.to(device), y.to(device) 
        
        y_logits=model(X).squeeze()
        y_pred_probs = torch.sigmoid(y_logits)
        y_preds = torch.round(y_pred_probs)
            
        test_loss += loss_fn(y_logits, y).item()
        test_acc += accuracy_fn(y_true=y.detach(),y_pred=y_preds.detach())
    
    test_loss /= len(data_loader)
    test_acc /= len(data_loader)
    print(f"Test loss: {test_loss:.5f} | Test accuracy: {test_acc:.2f}%\n")
    return test_loss

In [73]:
torch.manual_seed(42)
epochs=30
early_stopping = EarlyStopping()
for epoch in range(epochs):
  print("epoch :",epoch+1)
  train=train_step(model=mymodel,
             data_loader=train_dataloader_word2vec,
             loss_fn=loss_fn,
             optimizer=optimizer,
             accuracy_fn=accuracy_fn)
  
  test=test_step(model=mymodel,
            data_loader=test_dataloader_word2vec,
            loss_fn=loss_fn,
            accuracy_fn=accuracy_fn)

  if early_stopping(test):
      print(f"Training halted at epoch {epoch + 1}")
      break

epoch : 1
Train loss: 0.51698 | Train accuracy: 71.63%
Test loss: 0.35527 | Test accuracy: 84.26%

epoch : 2
Train loss: 0.34208 | Train accuracy: 85.22%
Test loss: 0.33615 | Test accuracy: 85.32%

epoch : 3
Train loss: 0.32967 | Train accuracy: 85.78%
Test loss: 0.31849 | Test accuracy: 85.92%

epoch : 4
Train loss: 0.32051 | Train accuracy: 86.33%
Test loss: 0.32421 | Test accuracy: 86.10%

epoch : 5
Train loss: 0.31505 | Train accuracy: 86.45%
Test loss: 0.31434 | Test accuracy: 86.32%

epoch : 6
Train loss: 0.31168 | Train accuracy: 86.68%
Test loss: 0.31933 | Test accuracy: 86.03%

epoch : 7
Train loss: 0.30886 | Train accuracy: 86.66%
Test loss: 0.31661 | Test accuracy: 86.09%

epoch : 8
Train loss: 0.30670 | Train accuracy: 86.83%
Test loss: 0.31097 | Test accuracy: 86.71%

Training halted at epoch 8


# Conclusion

In [11]:
import pandas as pd
from tabulate import tabulate

data = {
    "Model": ["MultinomialNB", "GaussianNB", "Random Forest", "XGBoost", "CatBoost", "LightGBM", "ANN", "LSTM"],
    "BoW": [0.8437, 0.7137, 0.8432, 0.8511, 0.8710, 0.8561, 0.8498,None ],
    "TF-IDF": [0.8531, 0.7770, 0.8428, 0.8541, 0.8662, 0.8566, 0.8648, None],  
    "Word2Vec": [None, 0.7544, 0.8255, 0.8436, 0.8574, 0.8455, 0.8509, 0.8671] 
}

df = pd.DataFrame(data)

print(tabulate(df, headers="keys", tablefmt="grid"))


+----+---------------+----------+----------+------------+
|    | Model         |      BoW |   TF-IDF |   Word2Vec |
+====+===============+==========+==========+============+
|  0 | MultinomialNB |   0.8437 |   0.8531 |   nan      |
+----+---------------+----------+----------+------------+
|  1 | GaussianNB    |   0.7137 |   0.777  |     0.7544 |
+----+---------------+----------+----------+------------+
|  2 | Random Forest |   0.8432 |   0.8428 |     0.8255 |
+----+---------------+----------+----------+------------+
|  3 | XGBoost       |   0.8511 |   0.8541 |     0.8436 |
+----+---------------+----------+----------+------------+
|  4 | CatBoost      |   0.871  |   0.8662 |     0.8574 |
+----+---------------+----------+----------+------------+
|  5 | LightGBM      |   0.8561 |   0.8566 |     0.8455 |
+----+---------------+----------+----------+------------+
|  6 | ANN           |   0.8498 |   0.8648 |     0.8509 |
+----+---------------+----------+----------+------------+
|  7 | LSTM   